In [ ]:
!pip install gensim
!pip install tensorflow
!pip install tf-keras
!pip install datasets
!pip install evaluate
import argparse
import json
import string
import random
import re
import os
import string
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from google.colab import drive
import evaluate


In [ ]:
# !pip install torch==2.7.0 'torch_xla[tpu]==2.7.0'

In [ ]:
# !pip list | grep torch

In [ ]:
# !add-apt-repository ppa:graphics-drivers/ppa
# !apt update
# !apt install nvidia-384 nvidia-384-dev
# !apt-get install g++ freeglut3-dev build-essential libx11-dev libxmu-dev libxi-dev libglu1-mesa libglu1-mesa-dev
# !pip install torch torchvision torchaudio torch_xla -f https://download.pytorch.org/whl/torch_stable.html
# !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
# !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/patches/1/cuda-repo-ubuntu1710-9-2-local-cublas-update-1_1.0-1_amd64
# !dpkg -i cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
# !apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
# !apt-get update
# !apt-get install cuda
# !dpkg -i cuda-repo-ubuntu1710-9-2-local-cublas-update-1_1.0-1_amd64
# !apt-get install nvidia-cuda-toolkit
# !pip install torch torchvision torchaudio
# import torch

In [ ]:
# # !pip3 uninstall torch torchvision torchaudio
# # !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
# import torch
# print(torch.cuda.is_available())

In [ ]:
drive.mount('/content/drive')
# Specifying the File Path
train_data_dir = '/content/drive/My Drive/641/train.jsonl'
val_data_dir = '/content/drive/My Drive/641/val.jsonl'
test_data_dir = '/content/drive/My Drive/641/test.jsonl'
output_dir = '/content/drive/My Drive/641/classification_result.jsonl'

Mounted at /content/drive


In [ ]:
def load_preprocess_data(file_path, test_flag):
    with open(file_path, 'r', encoding="utf8") as inp:
        # Read input data
        data = {
            'id': [],
            'title': [],
            'text': [],
            'label': [],
            'target': []
        }

        # extract data from jsonl file
        for i in inp:
            i = json.loads(i)
            if test_flag:
                data['id'].append(i['id'])
            else:
                data['id'].append(''.join(i['postId']))
            data['title'].append(''.join(i['postText']))
            data['text'].append(''.join(i['targetParagraphs']))

            if not test_flag:
                data['label'].append(''.join(i['tags']))
                data['target'].append(''.join(i['spoiler']))

        # assign corresponding value to input, output & id
            # clean text
        texts = [preprocess_text(text) for text in data['text']]
        titles = [preprocess_text(title) for title in data['title']]
        if not test_flag:
          labels = [preprocess_text(label) for label in data['label']]
          le = LabelEncoder()
          labels = le.fit_transform(labels)


        if debug:
            print(f"Sample spoiler: {labels[0:5]}")

        if not test_flag:
          data_dict = {
              "text" : titles,
              "label" : labels
          }
        else: # for test set
          data_dict = {
              "text" : texts,
          }

        dataset = Dataset.from_dict(data_dict)

    return dataset

In [ ]:
def preprocess_text(text):

    # Convert to lowercase
    text = text.lower()
    # # Remove special characters (keep only letters, numbers, spaces & some special punctuations)
    text = re.sub(r'[^a-zA-Z0-9\s,\!\"\']', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())

    if stopword_removal:
        # remove stop words
        tokenised = tokenize(text)
        sw = load_stopword()
        cleaned = remove_stopwords(tokenised, sw)
        text = " ".join(token for token in cleaned)

    return text
def tokenize(text):
    """Tokenize the input text by removing punctuation and splitting into words."""

    tokenized_text = []

    for word in re.findall(r'\w+', text):
        tokenized_text.append(word)

    return tokenized_text


def load_stopword():
    res = []
    if stopword_ref == 'nltk':
        res = stopwords.words('english')
    else:
        stopword_set = []
        with open('../stopwords.txt', 'r', encoding='utf-8') as f:
            for line in f:
                stopword_set.append(line.strip())

        for item in stopword_set:
            for word in re.findall(r'\w+', item):
                # Key: \w matches any alphanumeric character and underscore
                res.append(word)
        res = set(res)
        # if debug:
        #     print(res)
    return res


def remove_stopwords(tokens, stopwords):

    # reduce word to simplest root form
    porter = PorterStemmer()
    cleaned_tokens = [porter.stem(word) for word in tokens if word.lower() not in stopwords]

    return cleaned_tokens


def save_model(model, filename):
    """Save the trained model to a file."""
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    if debug:
        print(f"Model saved to {filename}")




In [ ]:
# for testing bert
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, pipeline
id2label = {0: "multi", 1: "passage", 2:'phrase'}
label2id = {"multi": 0, "passage": 1, 'phrase': 2}

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
debug = True
retrain = True
stopword_removal = False  # True, False
stopword_ref = 'nltk'  # nltk, pre
ref_x = 'text'  # title, text

nltk.download('stopwords')

# model_name = 't5-base'  # t5-small, t5-base, t5-large
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)

LEARNING_RATE = 5e-3
TRAIN_BATCH_SIZE = 64
VAL_BATCH_SIZE = 64
NUM_EPOCHS = 3
WEIGHT_DECAY = 0.005
LOGGING_STEPS = 10

# LEARNING_RATE = 3e-4
# TRAIN_BATCH_SIZE = 8
# VAL_BATCH_SIZE = 8
# NUM_EPOCHS = 2
# WEIGHT_DECAY = 0.3
# LOGGING_STEPS = 10

# LEARNING_RATE = 8e-5
# TRAIN_BATCH_SIZE = 8
# VAL_BATCH_SIZE = 8
# NUM_EPOCHS = 5
# WEIGHT_DECAY = 0.1
# LOGGING_STEPS = 10
# question_prefix = "Question: what is the following inferring in passage:"

# LEARNING_RATE = 4e-5
# TRAIN_BATCH_SIZE = 8
# VAL_BATCH_SIZE = 8
# NUM_EPOCHS = 5
# WEIGHT_DECAY = 0.05
# LOGGING_STEPS = 10

# LEARNING_RATE = 2e-5
# TRAIN_BATCH_SIZE = 8
# VAL_BATCH_SIZE = 8
# NUM_EPOCHS = 3
# WEIGHT_DECAY = 0.01
# LOGGING_STEPS = 10

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Question: Find in passage, what is '{title}' inferring in passage? Passage: {text}
# Question: What is the key message that '{title}' is inferring in passage? Passage: {text}
# Current Best
# Question: What is the key spoiler that '{title}' is inferring in passage? Passage: {text}
f1_metric = evaluate.load("f1")

def preprocess_function(dataset):
    return tokenizer(dataset["text"], truncation=True)
def preprocess_test_function(text):
    return tokenizer(text, truncation=True)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1_metric.compute(predictions=predictions, references=labels, average="weighted")

In [ ]:
# clean text & return a dataset (w/ text, title, target)
train_dataset = load_preprocess_data(train_data_dir, test_flag=False)
val_dataset = load_preprocess_data(val_data_dir, test_flag=False)


# apply tokenization and preprocessing of dataset
tokenized_train = train_dataset.map(preprocess_function, batched = True)
tokenized_val = val_dataset.map(preprocess_function, batched = True)


if debug:
    print(f"Sample tokenized train input: {tokenized_train[0]['text']}")
    print(f"Sample tokenized train target: {tokenized_train[0]['label']}")

# =========================================================================
# training


training_args = TrainingArguments(
    output_dir= "results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate= LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size= VAL_BATCH_SIZE,
    num_train_epochs= NUM_EPOCHS,
    weight_decay= WEIGHT_DECAY,
    logging_dir= "logs",
    logging_steps= LOGGING_STEPS,
    load_best_model_at_end = True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

# Evaluate the model on the validation dataset
eval_results = trainer.evaluate()

# Print the evaluation results
print(f"Evaluation results: {eval_results}")

trainer.save_model("./clf")

# ==========================================================================

# testing
with open(test_data_dir, 'r', encoding="utf8") as inp, open(output_dir, 'w', encoding="utf8") as out:
    # Read input data
    data = {
        'id': [],
        'title': [],
        'text': [],
        'label': [],
        'target': []
    }

    # extract data from jsonl file
    for i in inp:
        i = json.loads(i)
        data['id'].append(i['id'])
        data['title'].append(''.join(i['postText']))
        data['text'].append(''.join(i['targetParagraphs']))

    texts = [preprocess_test_function(text) for text in data['text']]
    titles = [preprocess_test_function(title) for title in data['title']]
    id = [id for id in data['id']]

    # using title
    # inputs = [f"Question: Classify the following text. Passage: {title}" for title in titles]

    # # Tokenize inputs
    # model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    from transformers import pipeline

    classifier = pipeline("sentiment-analysis", model="./clf")
    prediction = classifier(data['title'])

    count = 0
    for i in range(0,400):
      if prediction[i]['label'] == 'LABEL_0':
        pred = 'multi'
      elif prediction[i]['label'] == 'LABEL_1':
        pred = 'passage'
      else:
        pred = 'phrase'
      if count == 0:
          print(f'input sample: {id[i]}')
          print(f'prediction sample: {pred}')
          count = 1
      out.write(json.dumps({'id': id[i], 'spoilerType': pred}) + '\n')
    # print(prediction[0:5])


    # # Write results to output file
    # for i in enumerate(prediction):
    #     if count == 0:
    #       print(f'input sample: {id}')
    #       print(f'prediction sample: {pred['label']}')
    #       count = 1
    #     out.write(json.dumps({'id': id[i], 'spoilerType': pred}) + '\n')

    # with open(output_dir, 'r') as f:
    #   for line in f:
    #       data = json.loads(line)
    #       predictions.append(data)

    #   # Create DataFrame
    #   df = pd.DataFrame(predictions)

    #   # Save as CSV
    #   df.to_csv('/content/drive/My Drive/641/task2_submission.csv', index=False)

Sample spoiler: [1 2 2 0 2]
Sample spoiler: [1 0 2 0 1]


Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Sample tokenized train input: wes welker wanted dinner with tom brady, but patriots qb had better idea
Sample tokenized train target: 1


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wmy308-2 (wmy308-2-university-of-waterloo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,F1
1,1.022200,1.089305,0.214043
2,1.047000,1.069081,0.214043
3,1.034500,1.066354,0.233488


Evaluation results: {'eval_loss': 1.066353678703308, 'eval_f1': 0.23348754448398576, 'eval_runtime': 0.1106, 'eval_samples_per_second': 3617.752, 'eval_steps_per_second': 63.311, 'epoch': 3.0}


Device set to use cuda:0


input sample: 0
prediction sample: phrase
